In [ ]:
# Cell 1: ultralytics 설치
!pip install ultralytics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 48.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 125.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 97.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 67.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 17.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 112.3 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstallin

In [ ]:
# Cell 2: Drive 마운트
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Cell 3: 추론 대상 압축 해제
import os, zipfile

zip_path = "/content/drive/MyDrive/Colab Notebooks/Movon/dataset/unlabeled_seatbelt"
unzip_output = "/content/unlabeled_infer"
os.makedirs(unzip_output, exist_ok=True)

# zip 파일 하나 혹은 여러 개 모두 해제
for fname in os.listdir(zip_path):
    if fname.endswith(".zip"):
        with zipfile.ZipFile(os.path.join(zip_path, fname), 'r') as zip_ref:
            zip_ref.extractall(unzip_output)

print(f"✅ 압축 해제 완료: {unzip_output}")

✅ 압축 해제 완료: /content/unlabeled_infer


In [ ]:
import os
from glob import glob

# 압축 풀린 루트 확인
print("루트 폴더 내 디렉토리:")
print(os.listdir("/content/unlabeled_infer"))

# 이미지 경로 확인 (재귀적으로 탐색)
img_files = glob("/content/unlabeled_infer/**/*.jpg", recursive=True)
print(f"총 이미지 수: {len(img_files)}")

# 상위 몇 개만 출력
for path in img_files[:5]:
    print(path)

루트 폴더 내 디렉토리:
['train.txt', 'obj.data', 'obj_train_data', 'obj.names']
총 이미지 수: 10000
/content/unlabeled_infer/obj_train_data/20240524_002240_NOR_0035_0.jpg
/content/unlabeled_infer/obj_train_data/20240524_105956_NOR_0079_0.jpg
/content/unlabeled_infer/obj_train_data/20240523_233158_NOR_0074_0.jpg
/content/unlabeled_infer/obj_train_data/20240523_230322_NOR_0009_0.jpg
/content/unlabeled_infer/obj_train_data/20240523_203332_NOR_0002_0.jpg


In [ ]:
import os
import shutil
from glob import glob
from tqdm import tqdm
from ultralytics import YOLO
import cv2

# 1. 설정
model_path = "/content/drive/MyDrive/Colab Notebooks/Movon/runs/yolov8_seatbelt/weights/best.pt"
img_input_dir = "/content/unlabeled_infer/obj_train_data"  # 추론 대상 원본 이미지 폴더
save_dir = "/content/drive/MyDrive/Colab Notebooks/Movon/dataset/inference_result"  # ✅ 저장 위치 (Drive)

# 폴더 생성
os.makedirs(save_dir, exist_ok=True)

# 2. 모델 로드
model = YOLO(model_path)

# 3. 유효한 이미지 파일 리스트 생성 (에러 방지) -> 에러는 이미지 사이즈가 너무 많은 경우로, V2에서 정리.
img_files = glob(os.path.join(img_input_dir, "*.jpg"))
valid_img_files = [] # list로 추론하면 이미지가 너무 많아 RAM 다운
print("Checking image files for validity...")
for img_path in tqdm(img_files):
    try:
        img = cv2.imread(img_path)
        # Attempt a simple resize to check for issues
        img_resized = cv2.resize(img, (640, 640)) # Use a fixed size for checking
        valid_img_files.append(img_path)
    except Exception as e:
        print(f"Skipping invalid image: {img_path} due to error: {e}")

print(f"Found {len(valid_img_files)} valid images out of {len(img_files)}")


# 4. 추론 실행
results = model.predict(
    source=img_input_dir, # Use the list of valid images
    conf=0.5,
    save_txt=True,
    # save_conf=True,
    save=True,
    imgsz=640,
    project=save_dir,
    name="seatbelt_yolo_export",
    # exist_ok=True
    stream=True #
)

Checking image files for validity...


100%|██████████| 10000/10000 [00:19<00:00, 515.18it/s]


Found 10000 valid images out of 10000


In [ ]:
# Cell 3: 누락된 .txt 라벨 자동 생성
import glob

result_dir = os.path.join(save_dir, "seatbelt_yolo_export")
label_dir = os.path.join(result_dir, "labels")
image_dir = result_dir

# 이미지 리스트 수집
img_files = glob.glob(os.path.join(image_dir, "*.jpg"))

# 누락된 라벨 생성
for img_path in img_files:
    base = os.path.splitext(os.path.basename(img_path))[0]
    txt_path = os.path.join(label_dir, base + ".txt")
    if not os.path.exists(txt_path):
        open(txt_path, 'w').close()

print("✅ 누락된 .txt 파일 생성 완료")

✅ 누락된 .txt 파일 생성 완료


In [ ]:
from glob import glob
from PIL import Image
from collections import Counter
import numpy as np

# 🔍 이미지 폴더 경로 설정 (필요시 수정)
image_dir = "/content/unlabeled_infer/obj_train_data"

# 🔍 지원되는 이미지 확장자
img_exts = ['.jpg', '.jpeg', '.png']

# 이미지 파일 전체 수집
img_files = []
for ext in img_exts:
    img_files.extend(glob(f"{image_dir}/**/*{ext}", recursive=True))

# 이미지 해상도 수집
sizes = []
for path in img_files:
    try:
        with Image.open(path) as img:
            sizes.append(img.size)  # (width, height)
    except:
        print(f"⚠️ 이미지 열기 실패: {path}")

# 🎯 통계 출력
print(f"총 이미지 수: {len(sizes)}")

# 가장 흔한 해상도 top 5
common_sizes = Counter(sizes).most_common(5)
print("\n📊 가장 흔한 해상도 top 5:")
for size, count in common_sizes:
    print(f"{size[0]}x{size[1]} → {count}장")

# 평균 해상도
widths, heights = zip(*sizes)
avg_w, avg_h = np.mean(widths), np.mean(heights)
print(f"\n📏 평균 해상도: {int(avg_w)} x {int(avg_h)}")

총 이미지 수: 10000

📊 가장 흔한 해상도 top 5:
416x228 → 75장
458x234 → 66장
410x232 → 54장
425x239 → 51장
411x230 → 51장

📏 평균 해상도: 552 x 259


In [ ]:
# Cell X: YOLO 라벨 경계 확장 후처리 (비율 기준)
import os
from glob import glob

# 기존 YOLO 라벨 폴더
label_dir = "/content/drive/MyDrive/Colab Notebooks/Movon/dataset/inference_result/seatbelt_yolo_export/labels"
# 수정된 라벨 저장 폴더
output_dir = "/content/drive/MyDrive/Colab Notebooks/Movon/dataset/inference_result/seatbelt_yolo_export/labels_corrected"
os.makedirs(output_dir, exist_ok=True)

margin = 0.008  # 300px 기준 기댓값 2px

def clamp(x):
    return max(0.0, min(1.0, x))

for txt_file in glob(os.path.join(label_dir, "*.txt")):
    new_lines = []
    with open(txt_file, 'r') as f:
        for line in f:
            parts = line.strip().split()
            if len(parts) != 5:
                continue
            cls, x_c, y_c, w, h = map(float, parts)

            # YOLO → 좌표 변환
            x1 = x_c - w / 2
            x2 = x_c + w / 2
            y1 = y_c - h / 2
            y2 = y_c + h / 2

            # 경계 확장 적용
            if x1 < margin: x1 = 0.0
            if x2 > 1 - margin: x2 = 1.0
            if y1 < margin: y1 = 0.0
            if y2 > 1 - margin: y2 = 1.0

            # 다시 YOLO 포맷으로
            new_xc = (x1 + x2) / 2
            new_yc = (y1 + y2) / 2
            new_w = x2 - x1
            new_h = y2 - y1

            new_lines.append(f"{int(cls)} {clamp(new_xc):.6f} {clamp(new_yc):.6f} {clamp(new_w):.6f} {clamp(new_h):.6f}\n")

    out_path = os.path.join(output_dir, os.path.basename(txt_file))
    with open(out_path, 'w') as f:
        f.writelines(new_lines)

print("✅ YOLO 라벨 경계 확장 후처리 완료 (정규화 비율 기준)") # 대략 3분 소요

✅ YOLO 라벨 경계 확장 후처리 완료 (정규화 비율 기준)


In [ ]:
# Cell 5: 후처리된 .txt + 이미지 묶어서 zip 압축
from shutil import make_archive
import shutil
import os
from glob import glob

# 경로 설정
result_dir = "/content/drive/MyDrive/Colab Notebooks/Movon/dataset/inference_result/seatbelt_yolo_export"
label_dir = os.path.join(result_dir, "labels_corrected")
image_dir = result_dir

# export zip 폴더 준비
export_dir = os.path.join(result_dir, "export_zip_temp")
os.makedirs(export_dir, exist_ok=True)

# 이미지 리스트
img_files = glob(os.path.join(image_dir, "*.jpg"))

# 이미지 + 수정된 txt 복사
for img_path in img_files:
    base = os.path.splitext(os.path.basename(img_path))[0]
    txt_path = os.path.join(label_dir, base + ".txt")
    shutil.copy(img_path, os.path.join(export_dir, base + ".jpg"))
    shutil.copy(txt_path, os.path.join(export_dir, base + ".txt"))

# 압축
zip_path = os.path.join(result_dir, "seatbelt_yolo_export.zip")
make_archive(zip_path.replace(".zip", ""), 'zip', export_dir)

print("✅ 압축 완료:", zip_path) # 평균 10분 소요

✅ 압축 완료: /content/drive/MyDrive/Colab Notebooks/Movon/dataset/inference_result/seatbelt_yolo_export/seatbelt_yolo_export.zip


In [ ]:
# Cell 5: 다운로드
from google.colab import files
files.download(zip_path)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
# 필요 시 후처리만 초기화, 수치 변경 후 재작업

In [ ]:
# 후처리 초기화
import shutil
import os

# 🎯 경로 설정 (필요시 수정)
save_dir = "/content/drive/MyDrive/Colab Notebooks/Movon/dataset/inference_result"
corrected_label_dir = os.path.join(save_dir, "labels_corrected")
zip_file = os.path.join(save_dir, "seatbelt_yolo_export.zip")
zip_temp_dir = os.path.join(save_dir, "export_zip_temp")

# ✅ 1. labels_corrected 폴더 삭제
if os.path.exists(corrected_label_dir):
    shutil.rmtree(corrected_label_dir)
    print(f"🗑️ 삭제 완료: {corrected_label_dir}")
else:
    print(f"ℹ️ 존재하지 않음: {corrected_label_dir}")

# ✅ 2. zip 파일 삭제
if os.path.exists(zip_file):
    os.remove(zip_file)
    print(f"🗑️ 삭제 완료: {zip_file}")
else:
    print(f"ℹ️ 존재하지 않음: {zip_file}")

# ✅ 3. 임시 압축 폴더 삭제
if os.path.exists(zip_temp_dir):
    shutil.rmtree(zip_temp_dir)
    print(f"🗑️ 삭제 완료: {zip_temp_dir}")
else:
    print(f"ℹ️ 존재하지 않음: {zip_temp_dir}")

ℹ️ 존재하지 않음: /content/drive/MyDrive/Colab Notebooks/Movon/dataset/inference_result/labels_corrected
ℹ️ 존재하지 않음: /content/drive/MyDrive/Colab Notebooks/Movon/dataset/inference_result/seatbelt_yolo_export.zip
ℹ️ 존재하지 않음: /content/drive/MyDrive/Colab Notebooks/Movon/dataset/inference_result/export_zip_temp
